# Global Paramter

In [2]:
WORKING_DIR='/home/ec2-user/SageMaker'
SageMaker_ComfyUI = WORKING_DIR + '/SageMaker_ComfyUI'

In [3]:
%cd $WORKING_DIR

/home/ec2-user/SageMaker


## Test the comfyui API

In [4]:
from utils import comfyui_api
        
test_prompt = """
    (A_Rostov_Style:0.7), rough brush strokes:1.3, oil painting, soft lighting
    protrait, John Wick, shoreline, ocean, skyline, windy, daylight,
    soothing tones, calm colors, 
    hdr, (intricate details, hyperdetailed:1.15)
        """
seed = 4410130 # random.randint(1000000, 9999999)

comfyui_api.generate_clip(test_prompt, seed)


4410130
/home/ec2-user/SageMaker/workflows/workflow_api_txt2gif.json
8d9ba6b7-7ee7-48a5-bd7a-7992c3c533a1
/home/ec2-user/SageMaker/outputs/1_4410130.gif
/home/ec2-user/SageMaker/outputs/1_4410130.gif DONE!!!


# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb


In [5]:
# dependencies
!pip install --no-build-isolation --force-reinstall --quiet \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

# text
!pip install --quiet\
    langchain==0.0.309 \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client \
    apache-beam \
    datasets \
    tiktoken \
    "ipywidgets>=7,<8" \
    matplotlib 

# agents
!pip install --quiet \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    pysqlite3 \
    google-search-results

# entity extraction
!pip install --quiet beautifulsoup4

# image
!pip install --quiet "pillow>=9.5,<10"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.20.5 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
sparkmagic 0.20.5 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.0.3 which is incompatible.
sphinx 7.0.1 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [6]:

import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client()

#boto3_bedrock.list_foundation_models()


Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [7]:

from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


In [ ]:

from langchain.prompts import PromptTemplate
from utils import comfyui_api
import re, random

def genContent(task, input_variables, number, idea):
    LANGCHAIN_TASKS=SageMaker_ComfyUI + '/langchain_tasks'
    with open('{}/{}'.format(LANGCHAIN_TASKS, task), 'r', encoding="utf-8") as task:
        lines = task.readlines()
        # print(lines)
        # Create a prompt template that has multiple input variables
        multi_var_prompt = PromptTemplate(
            input_variables=input_variables, 
            template="""

        Human: {}

        Assistant: ok """.format(lines)
        )
        
        prompt = multi_var_prompt.format(number=1, idea=idea)
        
        num_tokens = textgen_llm.get_num_tokens(prompt)
        print(f"Our prompt has {num_tokens} tokens")
        
        response = textgen_llm(prompt)
        content = response[response.index('\n')+1:]
            
        # print(content)
        return content

def content2Prompt_1(task, content):
    indices = [i.start() for i in re.finditer('\n', content)]
    ideas = []
    for i in range(len(indices)):
        idea = ''
        if i < len(indices)-1:
            idea = content[indices[i]+1:indices[i+1]]
        else:
            idea = content[indices[i]+1:]
        # print(idea)
        ideas.append(idea)
    return ideas

def content2Prompt_2(task, content):
    indices = [i.start() for i in re.finditer('\n', content)]
    # print(indices)

    prompt_txt = ''
    if len(indices) < 2 :
        prompt_txt = content[indices[0]+1:]
    if len(indices) >= 2 :
        prompt_txt = content[indices[0]+1:indices[1]] 
    print(prompt_txt)

    return prompt_txt

def claude2comfyui(task, number, idea, idx):
    content = genContent(task, ["number", "idea"], number, idea)
    if len(content) == 0:
        return
    prompt = content2Prompt_2(task, content)
    # print(prompt)
    seed = random.randint(100000000, 999999999)
    comfyui_api.generate_clip(prompt, seed, idx)

task = 'story_prompt_generator.txt'
idea = 'John Wick cannot fall asleep'
content = genContent(task, ["number", "idea"], 10, idea)
ideas = content2Prompt_1(task, content)
print(ideas)

# ideas = [
#     'John Wick retires, wife dies, dog killed, seek revenge', 
#     'John visits Continental Hotel, meets old friend, requests help',
#     'John tracks down dog killer, fights henchmen, interrogates boss',
#     'John invades nightclub, shootout with guards, boss escapes',
#     'John chases boss, epic car chase, crashes, fights in subway'
#         ]

print("=======================")
task = 'midjourney_prompt_generator.txt'
i = 0
for idea in ideas:
    i = i+1
    print(idea)
    claude2comfyui(task, 1, idea, i)

# idea = 'John wick running in the steet'
# claude2comfyui(task, idea)
        

Our prompt has 357 tokens
['- JOHN WICK lying in bed staring at ceiling', '- Clock reads 3AM ', '- JOHN WICK gets up frustrated', '- JOHN WICK makes coffee', '- JOHN WICK drinks coffee looking out window ', '- City streets empty and quiet', '- JOHN WICK does pushups trying to tire himself', '- JOHN WICK takes shower ', '- JOHN WICK gets back in bed', '- JOHN WICK tosses and turns ', '- JOHN WICK takes sleeping pills', "- Pills don't work", '- JOHN WICK makes warm milk ', '- JOHN WICK drinks milk while watching TV', '- JOHN WICK dozes off on couch', '- JOHN WICK startles awake ', '- JOHN WICK punches pillow in frustration', '- Sun starts to rise ', '- JOHN WICK gives up and makes breakfast', '- JOHN WICK drinks more coffee looking tired', '- JOHN WICK starts his day exhausted']
- JOHN WICK lying in bed staring at ceiling
Our prompt has 1291 tokens
- John Wick resting in a dim bedroom, staring pensively at the ceiling, dramatic lighting, high contrast, black and white tones, cinematic pe